# 1.初始化环境
只考虑Colab环境, 不考虑本地NoteBook。

## 1.1 显卡驱动检查
查看CUDA版本以供PyTorch安装使用。

如果执行失败, 检查运行时中是否正确选择GPU。

## 1.2 PyTorch安装
[PyTorch官网](https://pytorch.org/)

暂时配置: Stable、Linux、Pip、Python、CUDA 11.3

使用官网给出的命令安装

## 1.3 挂载Google Drive
1. 不更改工作目录, 网络I/O通讯太慢。
2. 训练集直接拷贝到Colab本地。
3. **断点路径一定在Google Drive而不是Colab本地。**

以上问题在Config类中配置。

In [ ]:
# 1.1显卡驱动检查
!nvidia-smi

Tue Sep  6 16:57:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 1.2 PyTorch安装
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113


In [ ]:
# 1.3 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import time
import torch
from torch.nn import Module, LSTM, Linear, BatchNorm1d
from torch.utils.data import DataLoader, TensorDataset  # 常用工具区关于data区
from sklearn.model_selection import train_test_split

import os

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

import torch
import torchvision.models
from torchvision import transforms

# 2. 从网络硬盘中加载代码

## 2.1 路径的设置

In [ ]:
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive"
PROJECT_NAME = "fruit"
PROJECT_PATH = os.path.join(GOOGLE_DRIVE_PATH, PROJECT_NAME)

# - fruit
# +-- data
# +-- label


## 2.2 load_from_drive及其配套函数

### 2.2._
这个标题用于缩进

#### 2.2.1 标签原始数据的读取与分析
test_dict = load_labels_to_memomery(labels_path, is_preview=False)

In [ ]:
def load_labels_to_memomery(labels_path, is_preview=False):
    if not os.path.exists(labels_path):
        raise Exception("Labels path does not exist.")

    labels_dict = {}

    for SET in os.listdir(os.path.join(labels_path)):
        if SET[0] == ".":  # 跳过隐藏文件
            continue

        SET_name = SET.split(".")[0]
        if SET_name not in labels_dict:
            labels_dict[SET_name] = []
        read_from_file(os.path.join(labels_path, SET), labels_dict[SET_name])

    if is_preview:
        preview_labels(labels_dict)


    labels_dict_with_mean = {}
    for key in labels_dict:
        labels_dict_with_mean[key] = []
        for ls in labels_dict[key]:
            if ls[0] is None:
                labels_dict_with_mean[key].append(None)
            else:
                labels_dict_with_mean[key].append(np.mean(np.array(ls)))



    return labels_dict_with_mean



def read_from_file(file_path, ls):
    fl = open(file_path)

    while True:
        line = fl.readline()

        if line == "":
            break

        line_without_n = line.split("\n")[0]
        if line_without_n == "":
            continue

        commit = line_without_n.split("#")
        if len(commit) > 1:  # 被标记
            ls.append([None])  # 填None
            continue

        label_line = commit[0]
        index_and_labels = label_line.split(":")
        if len(index_and_labels) < 2:
            continue

        labels = index_and_labels[-1].split(" ")

        ls.append([])
        for label in labels:
            if label != "":
                ls[-1].append(float(label))

    fl.close()


def get_bar(ls, step, start=None, title=None):
    if start is None:
        start = int(min(ls))
    else:
        if start > int(min(ls)):
            print("指定的起点太大, 我还没写处理")
            exit(0)

    end = int(max(ls)) + 1

    names = [str(x) for x in range(start, end, step)]
    width = 0.75
    x_ticks = np.arange(len(names))
    # x_ticks一定要用numpy, 因为要做矩阵操作

    values = [0 for _ in range(len(names))]
    for var in ls:
        values[int((var - start) / step)] += 1

    # fig, ax = plt.subplots()
    # bar = plt.bar(names, values)
    bar = plt.bar(x_ticks + width / 2, values, width)  # 默认居中, 所以加1/2
    plt.xticks(x_ticks, names)  # 还原names
    plt.bar_label(bar, label_type="edge")
    if title is not None:
        plt.title(title)
    plt.show()


def analysis(ls, step=5, title=None):
    get_bar(ls, step, int(min(ls) / step) * step, title)


def preview_labels(labels_dict, step=5):
    all_data = [x
                for key in labels_dict
                for arr in labels_dict[key]
                for x in arr
                if x is not None]  # 我们将None异常算入
    avg_data = [sum(arr) / len(arr)
                for key in labels_dict
                for arr in labels_dict[key]
                if arr[0] is not None]
    analysis(all_data, step, "All Result Datas")
    analysis(avg_data, step, "Average Result per Fruit Datas")

#### 2.2.2 读取数据并随labels转化为numpy
test_dict = load_labels_to_memomery(labels_path, is_preview=False)

#### 2.2.2.1 用于缓存和检查是否可以直接从缓存里拿

In [ ]:
def load_pics_and_labels_to_numpy(datas_path, labels_dict):
    if not os.path.exists(datas_path):
        raise Exception("Datas path does not exist.")

    datas_list = []
    labels_list = []

    for DIR in os.listdir(os.path.join(datas_path)):
        if DIR[0] == ".":  # 跳过隐藏文件
            continue

        SET, LABEL = get_info_set_and_label(os.path.join(datas_path, DIR))
        # str, int SET = 1, 2, 3, 4... LABEL = 1, 2, 3, 4...
        if labels_dict[SET][LABEL - 1] is None:
            continue

        single_data_list = []
        for i in range(3):
            for j in range(6):
                video_name = str(i)
                pic_name = str(j).zfill(10) + ".jpg"
                single_data_list.append(single_img_to_numpy(os.path.join(
                    datas_path,
                    DIR,
                    video_name,
                    pic_name)))

        single_label_list = [labels_dict[SET][LABEL - 1]
                             for _ in range(6)]

        datas_list.append(np.array(single_data_list))
        labels_list.append(np.array(single_label_list))

    return np.array(datas_list), np.array(labels_list)


def single_img_to_numpy(img_path):
    trf = transforms.Compose([transforms.Resize((224, 224)),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    img = Image.open(img_path)
    img = trf(img)
    img = torch.unsqueeze(img, dim=0)
    net = torchvision.models.resnet18(pretrained=True)
    out = net(img)
    single_line_numpy = out.detach().cpu().numpy().reshape(-1)
    return single_line_numpy


def get_info_set_and_label(DIR_path):
    fl = open(os.path.join(DIR_path, "info.txt"))

    content = fl.readline()
    set_num = int(content.split("\n")[0].split(":")[-1])

    fl.readline()  # 空读

    content = fl.readline()
    label_num = int(content.split("\n")[0].split(":")[-1])

    fl.close()

    return str(set_num), label_num

#### 2.2.3 用于测试, 避免因小问题导致处理文件中间的耗时
直接运行, 与错误会报错

In [ ]:
def load_pics_and_labels_to_numpy_test(datas_path, labels_dict):
    if not os.path.exists(datas_path):
        raise Exception("Datas path does not exist.")
    
    for DIR in os.listdir(os.path.join(datas_path)):
        if DIR[0] == ".":  # 跳过隐藏文件
            continue

        SET, LABEL = get_info_set_and_label(os.path.join(datas_path, DIR))
        # str, int SET = 1, 2, 3, 4... LABEL = 1, 2, 3, 4...
        if labels_dict[SET][LABEL - 1] is None:
            continue
        for i in range(3):
            for j in range(6):
                video_name = str(i)
                pic_name = str(j).zfill(10) + ".jpg"
                single_img_to_numpy_test(os.path.join(
                    datas_path,
                    DIR,
                    video_name,
                    pic_name))
                
def single_img_to_numpy_test(img_path):
    if not os.path.exists(img_path):
        print(img_path)
        print("Oh shit")
        raise Exception("Image path does not exist.")

### 2.2 load_from_drive本体

In [ ]:
def load_from_drive(project_path):
    labels_dict = load_labels_to_memomery(os.path.join(project_path, "label"))
    load_pics_and_labels_to_numpy_test(os.path.join(project_path, "data"), labels_dict)
    return load_pics_and_labels_to_numpy(os.path.join(project_path, "data"), labels_dict)

### 2.3 得到数据集与标签集的numpy元组

```
dataset = train_data(r'E:\peach\data').reshape(-1, 18, 1000)  # reshape改为一个三维数组（-1是模糊控制，行未知，18列，1000页）
print(dataset.shape)
labelset = np.array(get_label(r'E:\peach\油桃硬度.xlsx'))
print(labelset.shape)
```

```
dataset, labelset = datas_and_labels
```

In [ ]:
datas_and_labels = load_from_drive(PROJECT_PATH)

# 3. 代码 

In [ ]:
print(datas_and_labels[0].shape)
print(datas_and_labels[1].shape)

(101, 18, 1000)
(101, 6)


## 3.1 三通道LSTM模型类

In [ ]:
class Lstm_model(Module):
    def __init__(self):  # 初始化
        super(Lstm_model, self).__init__()
        self.lstm1 = LSTM(input_size=1000, hidden_size=512,
                          num_layers=1, batch_first=True)
        self.lstm2 = LSTM(input_size=1000, hidden_size=512,
                          num_layers=1, batch_first=True)
        self.lstm3 = LSTM(input_size=1000, hidden_size=512,
                          num_layers=1, batch_first=True)
        self.linear1 = Linear(in_features=512, out_features=1)
        self.linear2 = Linear(in_features=512, out_features=1)
        self.linear3 = Linear(in_features=512, out_features=1)
        self.linear = Linear(in_features=18, out_features=1)
        self.active = torch.nn.LeakyReLU()
        self.bn = BatchNorm1d(num_features=18)

    def forward(self, x, hidden=None):
        x = x.reshape(1, -1, 1000)
        # print(x.shape)
        x01 = x[0, 0:6, :]
        x02 = x[0, 6:12, :]
        x03 = x[0, 12:18, :]
        x01 = x01.reshape(1, -1, 1000)
        x02 = x02.reshape(1, -1, 1000)
        x03 = x03.reshape(1, -1, 1000)
        x1, _ = self.lstm1(x01, hidden)
        x2, _ = self.lstm2(x02, hidden)
        x3, _ = self.lstm3(x03, hidden)
        # print(x1.shape)
        # print(x2.shape)
        # print(x3.shape)
        x1 = self.linear1(x1)
        x1 = self.active(x1)
        x2 = self.linear2(x2)
        x2 = self.active(x2)
        x3 = self.linear3(x3)
        x3 = self.active(x3)
        # print(x1.shape)
        # print(x2.shape)
        # print(x3.shape)
        x = torch.cat((x1, x2, x3), dim=1).reshape(-1)
        # print(x.shape)
        x = self.linear(x)
        # print(x.shape)
        return x

## 3.2 基本参数初始化

In [ ]:
device = torch.device('cuda:0')
model = Lstm_model().to(device)

dataset, labelset = datas_and_labels

train_X, valid_X, train_Y, valid_Y = train_test_split(dataset, labelset,
                                                      test_size=0.2, random_state=1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)  # 学习率
criterion = torch.nn.MSELoss()
train_X, train_Y = torch.from_numpy(train_X).float(), torch.from_numpy(train_Y).float()
train_loader = DataLoader(TensorDataset(train_X, train_Y), batch_size=1, shuffle=True)
valid_X, valid_Y = torch.from_numpy(valid_X).float(), torch.from_numpy(valid_Y).float()
valid_loader = DataLoader(TensorDataset(valid_X, valid_Y), batch_size=1, shuffle=True)


## 3.3 训练与评估

In [ ]:
def train(epoch_num):
    model.train()
    running_loss = []
    for index, (data, label) in enumerate(train_loader):
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(data)
        # label = torch.unsqueeze(label, dim=1)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        running_loss.append(loss.item())
    train_loss = np.mean(running_loss)
    model.eval()

    val_running_loss = []
    for index, (data, label) in enumerate(valid_loader):
        data, label = data.to(device), label.to(
            device)  # print(index)# print("label.shape", label.shape)# print("label", label)# print("data.shape", data.shape)
        optimizer.zero_grad()
        output = model(data)
        # label = torch.unsqueeze(label, dim=1)
        # print("output.shape:", output.shape)# print("output:", output)
        loss = criterion(output, label)  # print('loss:', loss)
        val_running_loss.append(loss.item())
    val_loss = np.mean(val_running_loss)
    print("epoch={}, train_loss:{}, val_loss:{}".format(epoch_num, train_loss, val_loss))
    return train_loss, val_loss  # 用于hook


def evl():
    # 预测过程
    model.eval()
    # hidden_predict = None
    trian_error_list = []

    for data, label in train_loader:
        data_X = data.to(device)
        # pred_X = model(data_X, hidden_predict)
        pred_X = model(data_X)
        cur_pred = pred_X.detach().cpu().numpy()
        cur_label = label.detach().cpu().numpy()
        # cur_label = torch.unsqueeze(label, dim=-1).detach().cpu().numpy()
        cur_erro = np.mean(np.abs(cur_pred - cur_label))
        trian_error_list.append(cur_erro)
    train_error = np.mean(trian_error_list, axis=0)

    val_error_list = []
    pre_list = []
    lab_list = []
    for data, label in valid_loader:
        data_X = data.to(device)
        pred_X = model(data_X)
        cur_pred = pred_X.detach().cpu().numpy().reshape(-1)
        cur_label = label.detach().cpu().numpy().reshape(-1)
        cur_erro = np.mean(np.abs(cur_pred - cur_label))
        val_error_list.append(cur_erro)
        pre_list.append(cur_pred)
        lab_list.append(cur_label)
    val_error = np.mean(val_error_list, axis=0)
    np.savetxt(os.path.join(PROJECT_PATH, 'pre.txt'),
                            np.array(pre_list).reshape(-1, 1),
                            fmt='%0.2f')
    np.savetxt(os.path.join(PROJECT_PATH, 'lab.txt'),
               np.array(lab_list).reshape(-1, 1), 
               fmt='%0.2f')
    # np.savetxt(r'E:\kiwi2\merge_kiwi\pre.txt', np.array(pre_list))
    # np.savetxt(r'E:\kiwi2\merge_kiwi\lab.txt', np.array(lab_list))
    print("train_error:{}, val_error:{}".format(train_error, val_error))
    return train_error, val_error

## 3.4 可视化

In [28]:
class DataViwer:
    def __init__(self, period=None, title=None):
        self.cnt = 0
        self.datas = []
        self.period = period
        self.title = title

    def add_data(self, new_data, new_period=None):
        self.cnt += 1
        self.datas.append(new_data)

        if new_period is not None:
            self.period = new_period

        if self.period is None:
            return

        if self.cnt != 0 and self.cnt % self.period == 0:
            self.show()

    def show(self):
        plt.cla()
        plt.plot(np.arange(len(self.datas)), np.array(self.datas))
        if self.title is not None:
            plt.title(self.title)
        plt.show()

# 4. main函数

In [ ]:
errormin = 0.9

tl = DataViwer(20, 'Loss/train')
vl = DataViwer(20, 'Loss/val')
te = DataViwer(20, 'Error/train')
ve = DataViwer(20, 'Error/val')


for epoch in range(1, 2000):
    print('epoch = ', epoch)
    train_loss, val_loss = train(epoch)
    train_error, val_error = evl()

    #writer.add_scalar('Loss/train', train_loss, epoch)
    #writer.add_scalar('Loss/val', val_loss, epoch)
    #writer.add_scalar('Error/train', train_error, epoch)
    #writer.add_scalar('Error/val', val_error, epoch)

    tl.add_data(train_loss)
    vl.add_data(val_loss)
    te.add_data(train_error)
    ve.add_data(val_error)

    error = val_error


    if error < errormin:
        errormin = error
        # time.sleep(10)
        torch.save(model.state_dict(), 
                   os.path.join(PROJECT_PATH, "weight", 'kiwi_new.pth'))
    print('-' * 80)

#print("end")

tl.show()
vl.show()
te.show()
ve.show()


print("finally_average_error:", errormin)
#writer.close()